In [1]:
import pandas as pd
import pathlib
import geopandas as gpd
import matplotlib.pyplot as plt
import pypsa
import numpy as np
import seaborn as sbn
import cartopy.crs as ccrs
pd.options.display.max_columns = None
pd.options.display.max_rows = None

/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/networkclustering.py:23: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


# Load the input data

In [2]:
base_path = pathlib.Path("/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/")
base_network_pypsa_earth = pypsa.Network(pathlib.Path(base_path, "geothermal-modelling/workflow/pypsa-earth/networks/US_2021/base.nc"))
base_network_pypsa_usa = pypsa.Network(pathlib.Path(base_path, "pypsa-usa/workflow/resources/Default/usa/elec_base_network.nc"))
eia_base_network = gpd.read_file(pathlib.Path(base_path, "geothermal-modelling/analysis/data/US_Electric_Power_Transmission_Lines_5037807202786552385.geojson"))

/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatibl

In [3]:
eia_base_network.head(5)

,OBJECTID_1,OBJECTID,ID,TYPE,STATUS,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,OWNER,VOLTAGE,VOLT_CLASS,INFERRED,SUB_1,SUB_2,SHAPE__Len,GlobalID,geometry
0,1,1,100511,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 861, EIA 860","Tue, 08 Nov 2016 00:00:00 GMT",Imagery,"Mon, 27 Mar 2017 00:00:00 GMT",Not Available,115.0,100-161,Yes,Waters River,Tap136228,79.523416,f1500a3a-804a-4da4-b6f6-bd0fbb2cf36d,"LINESTRING (-70.92855 42.54359, -70.92841 42.5..."
1,2,2,140809,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 860","Wed, 16 Apr 2014 00:00:00 GMT",Imagery,"Wed, 15 Feb 2017 00:00:00 GMT",Kingsport Power Co,161.0,100-161,Yes,Phillips Bend,John Sevier,24018.513114,58c24e54-d758-4071-9ebf-d83beb60c707,"LINESTRING (-82.96123 36.37699, -82.96090 36.3..."
2,3,3,140837,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, https://www9.nationalgridus.com/oasis...","Tue, 16 Jun 2015 00:00:00 GMT",Imagery,"Tue, 05 Mar 2019 00:00:00 GMT",Niagara Mohawk Power Corp.,115.0,100-161,Yes,Tap140359,Tap140373,5972.919614,c40bbff1-b1ed-4905-9ff6-1a80b7b422fc,"LINESTRING (-75.33407 43.08946, -75.33426 43.0..."
3,4,4,140811,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, OpenStreetMap","Fri, 20 Jun 2014 00:00:00 GMT",Imagery,"Mon, 20 Mar 2017 00:00:00 GMT","Entergy New Orleans, Llc",115.0,100-161,Yes,Gentilly Road,Michoud Station,14253.092823,f09483e3-5419-4b94-aac5-a6450f2452c4,"LINESTRING (-89.93728 30.00905, -89.93773 30.0..."
4,5,5,140813,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 860","Tue, 04 Oct 2016 00:00:00 GMT",Imagery/other,"Wed, 09 May 2018 00:00:00 GMT",Midamerican Energy Co,161.0,100-161,Yes,Unknown137689,Tap137690,8950.844317,36ac4f5e-ad18-47c2-bb7d-8ba64b7fa822,"LINESTRING (-90.56609 41.61256, -90.56468 41.6..."


The total number of lines in the EIA dataset is

In [3]:
eia_base_network.shape[0]

94216

At first glance, it is possible to see that some transmission lines correspond to a voltage of -999999.0 kV. In particular the number of lines corresponding to -999999.0 is

In [4]:
eia_base_network.loc[eia_base_network["VOLTAGE"]==-999999.0].shape[0]

14062

which acoounts for almost 15 % of the lines in the EIA dataset

In [5]:
eia_base_network.loc[eia_base_network["VOLTAGE"]==-999999.0].shape[0]/eia_base_network.shape[0]*100

14.9252780844018

For the moment, such lines are filtered out

In [6]:
eia_base_network = eia_base_network.loc[eia_base_network["VOLTAGE"]!=-999999.0]
eia_base_network.shape[0]

80154

Get the voltage bins from the reference data

In [7]:
eia_voltage_bins_flags = eia_base_network["VOLT_CLASS"].unique().tolist()
eia_voltage_bins_flags

['100-161',
 'Under 100',
 '345',
 '220-287',
 '500',
 '735 And Above',
 'Dc',
 'Not Available']

In [8]:
volt_class_min_values = []
volt_class_max_values = []
for bin_name in eia_voltage_bins_flags:
    volt_class_min_values.append(eia_base_network.loc[eia_base_network["VOLT_CLASS"]==bin_name]["VOLTAGE"].min())
    volt_class_max_values.append(eia_base_network.loc[eia_base_network["VOLT_CLASS"]==bin_name]["VOLTAGE"].max())
volt_class_bin_limits = pd.DataFrame(
    {
        "voltage_class": eia_voltage_bins_flags,
        "voltage_class_min": volt_class_min_values,
        "voltage_class_max": volt_class_max_values,
    }
)
volt_class_bin_limits

,voltage_class,voltage_class_min,voltage_class_max
0,100-161,100.0,218.0
1,Under 100,3.0,92.0
2,345,345.0,345.0
3,220-287,220.0,287.0
4,500,500.0,500.0
5,735 And Above,765.0,765.0
6,Dc,200.0,1000.0
7,Not Available,169.0,169.0


The voltage classes in the EIA dataset include also *Dc*. The number of lines corresponding to such voltage class is

In [9]:
eia_base_network.loc[eia_base_network["VOLT_CLASS"]=='Dc'].shape[0]

9

Considering that the PyPSA-Earth base network does not contain any DC line

In [10]:
base_network_pypsa_earth.lines["dc"].unique()

array([False])

the lines corresponding to the *Dc* voltage class are removed from the EIA dataset

In [11]:
eia_base_network = eia_base_network.loc[eia_base_network["VOLT_CLASS"]!='Dc']
eia_base_network.shape[0]

80145

Furthermore, the lines corresponding to the voltage class *Not Available* are removed as well

In [12]:
eia_base_network = eia_base_network.loc[eia_base_network["VOLT_CLASS"]!='Not Available']
eia_base_network.shape[0]

80144

In [13]:
eia_base_network["STATUS"].unique()

array(['In Service', 'Not Available', 'Under Construction', 'Inactive',
       'Proposed'], dtype=object)

# PyPSA-Earth - EIA comparison

Add the voltage class to the PyPSA-Earth base network

In [ ]:
base_network_pypsa_earth.lines["v_nom_class"] = base_network_pypsa_earth.lines["v_nom"]
base_network_pypsa_earth.lines

In [ ]:
v_nom_class_dict_pypsa_earth = {
    55.: 'Under 100',
    57.1: 'Under 100', 
    60.: 'Under 100',
    66.: 'Under 100',  
    69.: 'Under 100',  
    70.: 'Under 100',  
    88.: 'Under 100', 
    92.: 'Under 100', 
    100.: "100-161",
    115.: "100-161", 
    120.: "100-161",  
    125.: "100-161",  
    138.: "100-161",  
    160.: "100-161",  
    161.: "100-161",
    220.: "220-287",
    230.: "220-287",
    287.: "220-287",
    345.: "345",
    500.: "500",
    765.: "735 And Above"
}

In [ ]:
base_network_pypsa_earth.lines["v_nom_class"] = base_network_pypsa_earth.lines["v_nom_class"].replace(v_nom_class_dict_pypsa_earth)
base_network_pypsa_earth.lines

### Count plot

In [ ]:
sbn.countplot(data=base_network_pypsa_earth.lines, x=base_network_pypsa_earth.lines["v_nom_class"]).set(title='PyPSA-Earth base network')

In [ ]:
sbn.countplot(data=eia_base_network, x=eia_base_network["VOLT_CLASS"], color="orange").set(title='EIA network topology')

### Maps

In [ ]:
# Available voltage classes are:
# Under 100
# 100-161
# 220-287
# 345
# 500
# 735 And Above

selected_voltage_class_earth = "735 And Above"

In [ ]:
base_network_pypsa_earth.lines["line_width"] = 0.0
base_network_pypsa_earth.lines.loc[base_network_pypsa_earth.lines["v_nom_class"] == selected_voltage_class_earth, "line_width"] = 1.0

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, subplot_kw={"projection":ccrs.PlateCarree(), "projection":ccrs.PlateCarree()}, figsize=(20, 4))
base_network_pypsa_earth.plot(line_widths = base_network_pypsa_earth.lines["line_width"], bus_sizes = 0, line_colors = "blue", ax=ax1)
base_network_pypsa_earth.plot(line_widths = base_network_pypsa_earth.lines["line_width"], bus_sizes = 0, line_colors = "white", ax=ax2)
eia_base_network.loc[eia_base_network["VOLT_CLASS"]==selected_voltage_class_earth].plot(ax=ax2, color="orange")
fig.suptitle("Comparison for voltage class: {}".format(selected_voltage_class_earth))
ax1.set_aspect('equal')
ax1.title.set_text("PyPSA-Earth base network")
ax2.set_aspect("equal")
ax2.title.set_text("EIA network")

In [ ]:
fig.savefig("network_comparison_for_voltage_class_{}.png".format(str(selected_voltage_class_earth)))

# PyPSA-USA - EIA comparison

Add the voltage class to the PyPSA-USA base network

In [ ]:
base_network_pypsa_usa.lines["v_nom_class"] = base_network_pypsa_usa.lines["v_nom"]
base_network_pypsa_usa.lines

In [ ]:
base_network_pypsa_usa.lines.v_nom.unique()

In [ ]:
v_nom_class_dict_pypsa_usa = {
    69.: 'Under 100',
    100.: "100-161",
    138.: "100-161", 
    115.: "100-161", 
    161.: "100-161",
    230.: "220-287",
    345.: "345",
    500.: "500",
    765.: "735 And Above"
}

In [ ]:
base_network_pypsa_usa.lines["v_nom_class"] = base_network_pypsa_usa.lines["v_nom_class"].replace(v_nom_class_dict_pypsa_usa)

### Maps

In [ ]:
# Available voltage classes are:
# Under 100
# 100-161
# 220-287
# 345
# 500
# 735 And Above

selected_voltage_class_usa = "735 And Above"

In [ ]:
base_network_pypsa_usa.lines["line_width"] = 0.0
base_network_pypsa_usa.lines.loc[base_network_pypsa_usa.lines["v_nom_class"] == selected_voltage_class_usa, "line_width"] = 1.0

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, subplot_kw={"projection":ccrs.PlateCarree(), "projection":ccrs.PlateCarree()}, figsize=(20, 4))
base_network_pypsa_usa.plot(line_widths = base_network_pypsa_usa.lines["line_width"], bus_sizes = 0, line_colors = "blue", ax=ax1)
base_network_pypsa_usa.plot(line_widths = base_network_pypsa_usa.lines["line_width"], bus_sizes = 0, line_colors = "white", ax=ax2)
eia_base_network.loc[eia_base_network["VOLT_CLASS"]==selected_voltage_class_usa].plot(ax=ax2, color="orange")
fig.suptitle("Comparison for voltage class: {}".format(selected_voltage_class_usa))
ax1.set_aspect('equal')
ax1.title.set_text("PyPSA-USA base network")
ax2.set_aspect("equal")
ax2.title.set_text("EIA network")

In [ ]:
fig.savefig("network_comparison_pusa_for_voltage_class_{}.png".format(str(selected_voltage_class_usa)))

## Compare crossings

In [ ]:
gadm_usa_json_path_level_1 = "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/geothermal-modelling/analysis/data/gadm41_USA_1.json"
gadm_gdp_usa_level_1 = gpd.read_file(gadm_usa_json_path_level_1)

In [ ]:
gadm_gdp_usa_level_1.head(2)

Apply a spatial join between the reference transmission lines topology from EIA and the GADM shapes (level 1)

In [ ]:
spatial_join_gadm_eia = eia_base_network.sjoin(gadm_gdp_usa_level_1, how="inner")
spatial_join_gadm_eia.head(2)

In [ ]:
spatial_join_gadm_eia[["ID", "GID_1", "VOLTAGE", "VOLT_CLASS", "ISO_1", "NAME_1"]].head(2)

Apply a spatial join between the transmission lines topology from the PyPSA-Earth base network and the GADM shapes (level 1)

In [ ]:
base_network_pypsa_earth_geopandas = gpd.GeoDataFrame(base_network_pypsa_earth.lines, geometry=gpd.GeoSeries.from_wkt(base_network_pypsa_earth.lines.geometry), crs="EPSG:4326")

In [ ]:
spatial_join_gadm_pypsa_earth = base_network_pypsa_earth_geopandas.sjoin(gadm_gdp_usa_level_1, how="inner")
spatial_join_gadm_pypsa_earth.head(5)

In [ ]:
spatial_join_gadm_pypsa_earth[["GID_1", "v_nom", "v_nom_class", "type", "num_parallel", "ISO_1", "NAME_1"]].head(2)